# Project 04 Data Cleaning

In [41]:
import pickle

In [92]:
merged_all_w_vha_slim_pickle = pickle.load( open( "merged_all_w_vha_slim.p", "rb" ) )

In [93]:
import re
import string

In [94]:
import pandas as pd

In [95]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 400)
pd.options.display.float_format = '{:,.10f}'.format

#### Create copies of the base dataframe

In [96]:
df_tweets_final = merged_all_w_vha_slim_pickle.copy()

In [97]:
df_tweets_final.shape

(219712, 9)

In [98]:
df_tweets_final.head(2)

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
66,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
67,"@VAVetBenefits https://t.co/EdaCJSlLFI Mr. Secretary McDonough: Good afternoon. As you can see from my attached documentation, I have been fighting this issue of changing my discharge due to systemic racism for over 2 years. https://t.co/w58TgWxkUF",1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0


In [120]:
df_tweets_final[df_tweets_final['tweet'].str.contains("pandemic")][:5]

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
86,Veterans_Benefits_Admin are we going to see disability claims catch up i understand coronavirus_pandemic ruined a lot but mine has been ongoing since july thanks,1361835874980401153,papalima120,🇺🇸Paul L🇺🇸,2021-02-16,19:32:28,1,0,0
230,leoshane theveteransfirm department_of_va disabled veteran that for the last years have been tormented by the department_of_va Veterans_Benefits_Admin housevetaffairs memphisvamc there has been much failure cue and oversight on my claims i’ve not gotten coronavirus_pandemic vaccine yet but i do have a thoracic aortic aneurysms,1361678095183454211,myvetbiz,MyVetBiz,2021-02-16,09:05:31,1,1,0
454,Veterans_Benefits_Admin vaveteranssvcs Veterans_Health_Admin maziehirono brianschatz why did veteransaffairshawaii keep all coronavirus_pandemic vaccines that were meant to be divided between guam amp cnmi guamcongressman are you aware of this veterans tweeps please retweet,1361543215782879233,tsoulrebel671,Tommie/SoulRebel671,2021-02-16,00:09:33,1,0,1
704,yesterday i went with fellow veteran so he could get coronavirus_pandemic vaccine at northportvamc never was so impressed in the system they created to get and shots no drama nygovcuomo nycmayor department_of_va Veterans_Benefits_Admin Veterans_Health_Admin,1360577851137929216,neilecollins,Neil Collins,2021-02-13,08:13:32,1,1,1
760,please Veterans_Benefits_Admin department_of_va allow vet spouses to receive coronavirus_pandemic vaccine at the same time the sponsor does multiple trips to different places is inefficient and takes more time presssec vp potus consolidate and crunch the virus,1360307023116664832,parkerpmp,Kevin C Parker,2021-02-12,14:17:22,1,1,0


#### Create pre-processing helper function
...details on the sub functions further below

In [115]:
def update_base_table_for_NMF(df):
    
    # Pre-process data
    df.tweet = df.tweet.map(no_rt_marker)
    df.drop_duplicates(subset='tweet',inplace=True)
    df['tweet'] = df.tweet.map(no_at_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    df['tweet'] = df.tweet.map(no_lb_symbol)    # replace with a ' ', vs the rest of the replaces which will replace with ''
    df['tweet'] = df.tweet.map(punc_lower)
    df['tweet'] = df.tweet.map(no_website)
    df['tweet'] = df.tweet.map(SSA).map(SSA2).map(VBA).map(VBA2).map(VHA).map(VHA2).map(veteran_words).map(va_words).map(va_words2)
    df['tweet'] = df.tweet.map(black_veterans_words)
    df['tweet'] = df.tweet.map(covid_words) 
    df['tweet'] = df.tweet.map(vaccine_words) 
    df['tweet'] = df.tweet.map(alphanumeric)
    df['tweet'] = df.tweet.apply(remove_emoji)
        
    return df.head()

In [116]:
update_base_table_for_NMF(df_tweets_final)

,tweet,id,username,name,date,time,VBA Tweet,VA Tweet,VHA Tweet
66,Veterans_Benefits_Admin mr secretary mcdonough good afternoon as you can see from my attached documentation i have been fighting this issue of changing my discharge due to systemic racism for over years,1361866444410155013,clickforjustice,clickforjustice,2021-02-16,21:33:57,1,0,0
70,Veterans_Benefits_Admin in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your …,1361854662488555525,lccc_sva,LCCC SVA,2021-02-16,20:47:08,1,0,0
74,Veterans_Benefits_Admin vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to …,1361854659409965057,lccc_sva,LCCC SVA,2021-02-16,20:47:07,1,0,0
78,in howard became the first female fourstar admiral amp the first to command operational forces making her the highest ranking woman amp africanamerican in navy history after a remarkable career adm howard retired in thank you for your service adm howard,1361842314264502273,vavetbenefits,Veterans Benefits,2021-02-16,19:58:04,1,0,0
82,vbatribute trailblazer michelle howard set many firsts during her career with usnavy a veteran of the gulf war she served in operation desert shield and desert storm amp in became the first africanamerican woman to command a navy ship the uss rushmore,1361842311911497731,vavetbenefits,Veterans Benefits,2021-02-16,19:58:03,1,0,0


#### Pickle Updated Dataframe

In [121]:
df_tweets_final.shape

(198669, 9)

In [122]:
pickle.dump(df_tweets_final, open("df_tweets_final_processed.p", "wb" ) )

In [123]:
df_tweets_final_processed = pickle.load( open( "df_tweets_final_processed.p", "rb" ) )

___

### ~ Data Cleaning Appendix ~

#### Remove Duplicate Tweets Retweet Markers

In [50]:
no_rt_marker = lambda x: re.sub(r"RT @", '',x)

In [51]:
#df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(no_rt_marker)

In [52]:
#df_tweets_2_c.drop_duplicates(subset='tweet',inplace=True)

#### Alpha Numeric and Punctuation Adjustments
Will leave alpha-numeric for first run

In [53]:
alphanumeric = lambda x: re.sub('\w*\d\w*', '', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x.lower())

In [54]:
# regex = r"(?<!\d)[.,;:](?!\d)"
# test_str = "This is a 1example of the text. But, it only is 2.5 percent of all data"
# remove = re.sub(regex, "", test_str, 0)

In [55]:
#df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(punc_lower)

#### Remove Websites

In [56]:
no_website = lambda x: re.sub(r"http\S+", '', x)

In [57]:
#df_tweets_2_c['tweet'] = df_tweets_2_c.tweet.map(no_website)

#### Remove Other Typical Twitter Elements (If Still Present)

In [58]:
no_at_symbol = lambda x: re.sub(r"@", ' ', x)

In [59]:
no_lb_symbol = lambda x: re.sub(r"#", ' ', x)

In [60]:
#remove_department_of_va = lambda x: re.sub(r"departmentofva", '', x)

#### Create Columns for VA, VBA and VHA Original Tweets

#### Replace Named Groups

In [100]:
ssa_word_list = ['social security', 'socialsecurity']
ssa_word_list2 = [' ss ',' ssa ']
vba_word_list = ['vavetbenefits','veterans benefits administration', 
                 'veteranbenefitsadministration']
vba_word_list2 = [' vba ']
vha_word_list = ['veteranshealth','veterans health administration']
vha_word_list2 = [' vha ']
veteran_word_list = ['i am a veteran','iamaveteran', 'i am a vet','im a vet']
va_word_list = ['deptvetaffairs']
va_word_list2 = ['the va ']
black_veterans = ['black veterans','black veteran','black vets','blackvets','blackveterans']
covid_list = ['covidpandemic','covid pandemic','covid','coronavirus','covid19','covid-19','covid 19','the pandemic','this pandemic']
vaccine_list = ['coronavirus_pandemicvaccine','covid19vaccine','covidvaccine','covid19 vaccine','covid-19 vaccine','covid19vaccination','covid vaccine','covid19 vaccine',
          'covid-19 vaccines','coronavirus_pandemic vaccine','coronavaccine', 'the vaccine']


In [101]:
#res = re.sub("|".join(sorted(ssa_word_list, key = len, reverse = True)), repl_wrd, test_str) 

In [102]:
SSA = lambda x: re.sub("|".join(sorted(ssa_word_list, key = len, reverse = True)), 
                       'Social_Security', x)

In [103]:
SSA2 = lambda x: re.sub("|".join(sorted(ssa_word_list2, key = len, reverse = True)), 
                       ' Social_Security ', x)

In [104]:
VBA = lambda x: re.sub("|".join(sorted(vba_word_list, key = len, reverse = True)), 
                       'Veterans_Benefits_Admin', x)

In [105]:
VBA2 = lambda x: re.sub("|".join(sorted(vba_word_list2, key = len, reverse = True)), 
                       ' Veterans_Benefits_Admin ', x)

In [106]:
VHA = lambda x: re.sub("|".join(sorted(vha_word_list, key = len, reverse = True)), 
                       'Veterans_Health_Admin', x)

In [107]:
VHA2 = lambda x: re.sub("|".join(sorted(vha_word_list2, key = len, reverse = True)), 
                       ' Veterans_Health_Admin ', x)

In [108]:
veteran_words = lambda x: re.sub("|".join(sorted(veteran_word_list, key = len, reverse = True)), 
                       'i_am_a_veteran', x)

In [109]:
va_words = lambda x: re.sub("|".join(sorted(va_word_list, key = len, reverse = True)), 
                       'department_of_va', x)

In [110]:
va_words2 = lambda x: re.sub("|".join(sorted(va_word_list2, key = len, reverse = True)), 
                       'department_of_va ', x)

In [111]:
black_veterans_words = lambda x: re.sub("|".join(sorted(black_veterans, key = len, reverse = True)), 
                       'black_veterans', x)

In [112]:
covid_words = lambda x: re.sub("|".join(sorted(covid_list, key = len, reverse = True)), 
                       'coronavirus_pandemic', x)

In [113]:
vaccine_words = lambda x: re.sub("|".join(sorted(vaccine_list, key = len, reverse = True)), 
                       'coronavirus_pandemic vaccine', x)

#### Remove Emojis

In [114]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [76]:
remove_emoji('patience🤦‍ Veterans_Benefitsssss_Admin 🙏depar🙏tment_of_va Veterans_Health_Admin')
#emoji("Hi, I am Emoji  😜")

'patience Veterans_Benefitsssss_Admin department_of_va Veterans_Health_Admin'

#### Identify Stop Words

In [38]:
from sklearn.feature_extraction import text 

In [39]:
additional_stop_words = ['rt','vets','vet','veteran','veterans']
additional_stop_words2 = ['rt','vets','vet','veteran','veterans','Veterans_Benefits_Admin','veterans_benefits_admin', 'veterans_health_admin',
                          'Veterans_Health_Admin','department_of_va','veteransbenefitsadmin departmentofva', 
                          'veteransbenefitsadmin departmentofva veterans_health_adminadminadminadminadmin', 
                          'departmentofva veterans_health_adminadminadminadminadmin', 'veteransbenefitsadmin', 
                          'veterans_health_adminadminadminadminadmin', 'departmentofva','va']

stop_wordz = text.ENGLISH_STOP_WORDS.union(additional_stop_words)
stop_wordz2 = text.ENGLISH_STOP_WORDS.union(additional_stop_words2)